In [1]:
import re
import os
import pandas as pd
from string import punctuation as punkt
from collections import defaultdict

import sqlite3
db = sqlite3.connect('OHD.db')
cursor = db.cursor()

In [2]:
R_INDEX = defaultdict(lambda: defaultdict(set))
cursor.execute('SELECT * FROM Tokens')
T = cursor.fetchall()

In [3]:
T[0]

(8191, 1, 1, 0, 'Fuit', 'fuit', 'L', 'fuit', None)

In [56]:
TtoT = {}
for i in T:
    try:
        TtoT[(i[1], i[3])] = (i[4], re.sub('<.*?>', '', i[8]).replace('\n','<br>'))
    except:
        TtoT[(i[1], i[3])] = (i[4], i[8])

In [5]:
#"Tokens": 0 id_token, 1 id_text, 2 id_sent, 3 in_text, 4 token, 5 normalized, 6 lang 7 transitioned 8 dict
for i in T:
    R_INDEX[i[7]][i[1]].add(i[3])
    R_INDEX[i[5]][i[1]].add(i[3])

In [6]:
def find_docs(words):
    all_indexes = [{} for _ in words]
    if len(words) > 0:
        result_t = set(R_INDEX[words[0]])
        all_indexes[0] = R_INDEX[words[0]]
        for key, i in enumerate(words[1:]):
            result_t = result_t & set(R_INDEX[i])
            all_indexes[key+1] = R_INDEX[i]
    else:
        result_t = set()
        
    #print (list(i.keys() for i in all_indexes))
    all_indexes = [{i:t[i] for i in t if i in result_t} for t in all_indexes]
    return result_t, all_indexes

In [7]:
R_INDEX['ioseph'], R_INDEX['iosepf']

(defaultdict(set, {1: {710, 741, 1091}}),
 defaultdict(set, {1: {710, 741, 1091}}))

In [49]:
R_INDEX = dict(R_INDEX)

In [51]:
for i in R_INDEX:
    R_INDEX[i] = dict(R_INDEX[i])

In [8]:
find_docs(['got','ist'])

({1, 4, 10, 12},
 [{1: {1037, 1253, 1580},
   4: {20, 190, 453, 720, 1536, 1673, 1711},
   10: {1568, 1576},
   12: {151}},
  {1: {272,
    332,
    344,
    464,
    970,
    976,
    981,
    1124,
    1378,
    1418,
    1729,
    2022,
    2245,
    4084},
   4: {673,
    685,
    687,
    697,
    701,
    711,
    735,
    820,
    896,
    916,
    934,
    1086,
    1098,
    1307,
    1834,
    1905},
   10: {49,
    124,
    150,
    464,
    561,
    565,
    833,
    934,
    998,
    1257,
    1266,
    1278,
    1285,
    1330,
    1418,
    1516,
    1569,
    1773,
    1942,
    1979,
    2034},
   12: {262}}])

In [9]:
def search(docs, indexes, begin=-10, end=10):
    total = defaultdict(list)
    for key, value in enumerate(docs):
        variants = indexes[0][value]
        for i in variants:
            area = set(range(max(0, begin+i), i+end+1))
            result = set([i])
            k = 1
            for i in indexes[1:]:
                overlap = area & i[value]
                if len(overlap) > 0:
                    result.update(overlap)
                    k += 1
                else:
                    break
            if k == len(indexes):
                total[value].append(result)
    return dict(total)

In [10]:
search(*find_docs(['got','ist']), begin=-10, end=10)

{10: [{1568, 1569}, {1569, 1576}], 4: [{711, 720}]}

In [11]:
search(*find_docs(['got','ist']), begin=-20, end=20)

{10: [{1568, 1569}, {1569, 1576}], 4: [{701, 711, 720, 735}]}

In [57]:
TtoT[(4, 735)]

('ist',
 'list двн. свн. мр жр (i) искусство (перв. в военном деле), знание, наука; затем хитрость, коварство; нн. List; да. ди. list, го. lists.')

In [38]:
cursor.execute('SELECT id_text, title FROM Texts')
TEXTS = {i[0]:i[1] for i in cursor.fetchall()}
TEXTS

{1: 'Der althochdeutsche Tatian',
 2: 'Die Straßburger Eide',
 3: 'Das Wessobrunner Gebet',
 4: 'Der althochdeutsche Isidor',
 5: 'Der Erste Merseburger Zauberspruch',
 6: 'Der Zweite Merseburger Zauberspruch',
 7: 'Muspilli',
 8: 'Das Hildebrandslied',
 9: 'Kasseler Glossen (Kasseler Gespräche)',
 10: 'Otrfrids Evangelienbuch\t',
 11: 'Der Ältere Physyologus',
 12: 'Ludwigslied',
 13: 'Notker - Boethius Consolatio philosophiae'}

In [53]:
import json
import pickle as pkl

In [58]:
pkl.dump(TtoT, open('TtoT.pkl','wb'))
pkl.dump(T, open('T.pkl','wb'))
pkl.dump(R_INDEX, open('R_INDEX.pkl','wb'))
pkl.dump(TEXTS, open('TEXTS.pkl','wb'))

In [31]:
def interpreter(result):
    sentences = []
    for text in result:
        for item in result[text]:
            left = max(0, min(item) - 20)
            right = max(item) + 20
            sentence = ''
            for i in range(left, right+1):
                if (text, i,) in TtoT:
                    token = TtoT[(text, i,)]
                    if i in item:
                        tooltipped = '<a href="#" data-toggle="tooltip" title="{}"><b>{}</b> </a>'.format(token[1],token[0])
                        sentence = sentence + '<b>'+ tooltipped +'</b>' + ' '
                    else:
                        tooltipped = '<a href="#" data-toggle="tooltip" title="{}">{} </a>'.format(token[1],token[0])
                        sentence = sentence + tooltipped + ' '
            sentences.append((TEXTS[text], sentence,))
    return sentences

In [24]:
search(*find_docs(['got','ist']))

{10: [{1568, 1569}, {1569, 1576}], 4: [{711, 720}]}

In [33]:
def html_encoder(result):
    output = '<table border="2">'
    for i in result:
        output = output + '<tr><td>' + '</td><td>'.join(['<b>{}</b>'.format(i[0]), i[1]])+'</td></tr>'
    output += '</table>'
    return output

In [35]:
with open('doc.html','w') as f:
    R = interpreter(search(*find_docs(['ih'])))
    print (len(R))
    f.write(html_encoder(R))

51


In [30]:
interpreter(search(*find_docs(['ih'])))    

[('Der althochdeutsche Tatian',
  '<a href="#" data-toggle="tooltip" title="<heading><strong>asneri</strong></heading> <gender>мр</gender> (ja) <em>наемный рабочий, поден\xadщик</em>."">asnere </a> <a href="#" data-toggle="tooltip" title="<heading><strong>min</strong></heading>, <lang>свн.</lang> <lex>min</lex> (прит. мест.) <em>мой</em>; <lang>нн.</lang> <lex>mein</lex>; <lang>дс.</lang> <lex>min</lex>, <lang>да.</lang> <lex>гшп</lex>, <lang>ди.</lang> <lex>rninn</lex>; <lang>го.</lang> <lex>meins</lex>; <linked>см. Ш.</linked>.<br><heading><strong>min</strong></heading> <source>— Исид.—</source> (нар.), <lex>minnero</lex> <source>— Нотк.—</source> сравн., <linked>см. luzzil</linked>."">mines </a> <a href="#" data-toggle="tooltip" title="<heading><strong>fater</strong></heading> <lang>двн.</lang> <lang>свн.</lang> <gender>мр</gender> <em>отец</em>; <lang>нн.</lang> <lex>Vater</lex>; <lang>дс.</lang> <lex>fadar</lex>, <lang>да.</lang> <lex>fæder</lex>; <lang>ди.</lang> <lex>faðir</lex>

In [20]:
find_docs(['got','ist'])

({1, 4, 10, 12},
 [{1: {1037, 1253, 1580},
   4: {20, 190, 453, 720, 1536, 1673, 1711},
   10: {1568, 1576},
   12: {151}},
  {1: {272,
    332,
    344,
    464,
    970,
    976,
    981,
    1124,
    1378,
    1418,
    1729,
    2022,
    2245,
    4084},
   4: {673,
    685,
    687,
    697,
    701,
    711,
    735,
    820,
    896,
    916,
    934,
    1086,
    1098,
    1307,
    1834,
    1905},
   10: {49,
    124,
    150,
    464,
    561,
    565,
    833,
    934,
    998,
    1257,
    1266,
    1278,
    1285,
    1330,
    1418,
    1516,
    1569,
    1773,
    1942,
    1979,
    2034},
   12: {262}}])

In [ ]:
def 

In [37]:
import json
json.dumps('aƕa',ensure_ascii=False)

'"aƕa"'

In [ ]:
def find_docs(words):
    all_indexes = [{} for _ in words]
    if len(words) > 0:
        result_t = set(R_INDEX[words[0]])
        all_indexes[0] = R_INDEX[words[0]]
        for key, i in enumerate(words[1:]):
            result_t = result_t & set(R_INDEX[i])
            all_indexes[key+1] = R_INDEX[i]
    else:
        result_t = set()
        
    #print (list(i.keys() for i in all_indexes))
    all_indexes = [{i:t[i] for i in t if i in result_t} for t in all_indexes]
    return result_t, all_indexes

def search_doc(docs, indexes, begin=-10, end=10):
    total = defaultdict(list)
    for key, value in enumerate(docs):
        variants = indexes[0][value]
        for i in variants:
            area = set(range(max(0, begin+i), i+end+1))
            result = set([i])
            k = 1
            for i in indexes[1:]:
                overlap = area & i[value]
                if len(overlap) > 0:
                    result.update(overlap)
                    k += 1
                else:
                    break
            if k == len(indexes):
                total[value].append(result)
    return dict(total)

def interpreter(result):
    sentences = []
    for text in result:
        for item in result[text]:
            left = max(0, min(item) - 20)
            right = max(item) + 20
            sentence = ''
            for i in range(left, right+1):
                if (text, i,) in TtoT:
                    token = TtoT[(text, i,)]
                    if i in item:
                        if token[1] is not None:
                            tooltipped = '<a href="#" data-toggle="tooltip" title="{}"> <font size="5"> {}</font> </a>'.format(token[1],token[0])
                        else:
                            tooltipped = '<font size="5">{} </font>'.format(token[0])
                        sentence = sentence + '<b>'+ tooltipped.replace('\n','<br>') +'</b>' + ' '
                    else:
                        if token[1] is not None:
                            tooltipped = '<a href="#" data-toggle="tooltip" title="{}">{} </a>'.format(token[1],token[0])
                        else:
                            tooltipped = '{} '.format(token[0])
                        sentence = sentence + tooltipped.replace('\n','<br>') + ' '
            sentences.append((TEXTS[text], sentence,))
    return sentences

In [60]:
#"Tokens": 0 id_token, 1 id_text, 2 id_sent, 3 in_text, 4 token, 5 normalized, 6 lang 7 transitioned 8 dict
T[0]

(8191, 1, 1, 0, 'Fuit', 'fuit', 'L', 'fuit', None)

In [61]:
cursor.execute('SELECT * FROM Sentences')
S = cursor.fetchall()

In [86]:
S[0]

(1, 1, 'Cap. 2', 'Sentence: 1')

In [77]:
def norm(token, dic):
    if dic is not None:
        tooltipped = '<a href="#" data-toggle="tooltip" title="{}">{} </a>'.format(re.sub('<.*?>', '', dic).replace('\n','<br>'),token)
    else:
        tooltipped = '{} '.format(token)
    return tooltipped

In [154]:
t = 10
result = []
s = set()
for i in T:
    if i[1] == t:
        if i[2] not in s:
            for j in S:
                if j[0] == i[2]:
                    if j[2] == par or j[2] is None: 
                        par2 = ' '
                    else: 
                        par = j[2]
                        par2=par
                    sub = j[3]
                    if sub is None: sub = ' '
                    break
            result.append([par2, sub, norm(i[4], i[8])])
            s.add(i[2])
        else:
            result[-1][-1] = result[-1][-1] + norm(i[4], i[8])
            
html = '<table><tr><td width=10%>'
for i in result:
    html += i[0]
    html += '</td><td width=10%>'
    html += '</td><td>'.join(i[1:])
    html += '<td></tr><tr><td>'
html += '</td></tr></table>'
html = html.replace('\n','<br>')

In [151]:
TEXTS

{1: 'Der althochdeutsche Tatian',
 2: 'Die Straßburger Eide',
 3: 'Das Wessobrunner Gebet',
 4: 'Der althochdeutsche Isidor',
 5: 'Der Erste Merseburger Zauberspruch',
 6: 'Der Zweite Merseburger Zauberspruch',
 7: 'Muspilli',
 8: 'Das Hildebrandslied',
 9: 'Kasseler Glossen (Kasseler Gespräche)',
 10: 'Otrfrids Evangelienbuch\t',
 11: 'Der Ältere Physyologus',
 12: 'Ludwigslied',
 13: 'Notker - Boethius Consolatio philosophiae'}

In [159]:
with open ('./templates/evangelienbuch.html', 'w') as f:
    f.write('<style>\ntable {\nborder-style: no;\n}\n</style>{% extends "base.html" %}\n{% block content %}\n')
    f.write(html)
    f.write('\n{% endblock %}')